# Parameters

This section defines the parameter dataclass which is used for storing all the design data corresponding to the airplane. This class will be initialized before the start of every new analysis in the trade study. Below code block defines the class:

In [29]:
from dataclasses import dataclass

@dataclass
class AircraftParameters:
    """
        Data class defining aircraft parameters

        Note: Ensure values are provided in correct units
    """

    # parameters
    A: int
    S: float
    P_takeoff: float

    # Mission details
    range: int = 1200 # nmi
    endurance: float = 45/60 # hr

    # Density, slugs/cu ft
    rho_sea_level: float = 0.00237717
    rho_cruise: float = 0.00186850
    rho_loiter: float = 0.00211114
    dyn_viscosity: float = 3.78456e-7
    cruise_speed: float = 337.56 # ft/s
    cruise_mach_number: float = 0.3

    # Wing parameters
    taper_ratio_wing: float = 0.4
    tc_root: float = 0.18
    tc_tip: float = 0.09
    tc_avg: float = 0.16
    sweep_qc: float = 0.0

    # fuselage
    Sf: float = 380.12 # sq ft, fuselage wetted area
    Lf: float = 27 # ft, fuselage structural length without nose and tail cap
    D: float = 1.5/12 # ft, structural depth
    Lt: float = 16 # ft, tail moment arm

    # H-Tail
    Sht: float = 31 # sq ft
    sweep_ht_qc: float = 10 # deg
    taper_ratio_ht: float = 0.741

    # V-Tail
    Svt: float = 17 # sq ft
    sweep_vt_qc: float = 30 # deg
    taper_ratio_vt: float = 0.55
    vtail_factor: float = 0.0

    # Propulsion
    num_engines: int = 2
    num_fuel_tanks: int = 2
    prop_eff_cruise: float = 0.8
    prop_eff_loiter: float = 0.7
    Cbhp: float = 0.4 # lbs/hp-hr
    install_loss_factor: float = 0.08

    # Takeoff
    mu: float = 0.04
    mu_brakes: float = 0.5
    t_rotation: float = 1 # sec

    # landing
    gamma_approach: float = 3 #deg
    t_free_roll: float = 1 # sec

    # Weight fractions
    W1_W0: float = 0.985 # engine start, warmup, and takeoff
    W2_W1: float = 0.99 # climb
    W4_W3: float = 0.992 # descent
    W5_W4: float = 0.99 # climb
    W7_W6: float = 0.992 # descent
    W8_W7: float = 0.992 # landing

    # Other parameters
    Nz: float = 1.5 * 3.5 # limit load factor is set to 3.5
    avgas_density: float = 6.41 # lbs/gallon at -40 C
    num_passengers: float = 6 # including crew
    Kh: float = 0.05 # factor used in hydraulics weight estimation
    g: float = 32.17 # ft/s^2
    h_obs: float = 50 # ft
    PI: float = 3.14159

    def __post_init__(self):
        """
            Method to compute additional quantities based on the define data
        """

        taper_ratio = self.taper_ratio_wing
        
        self.e = 1.78 * (1 - 0.045*self.A**0.68) - 0.64

        # Compute wing planform
        self.b = (self.A * self.S)**0.5
        self.wing_root_chord = 2*self.S / self.b / (1 + taper_ratio) # ft
        self.wing_tip_chord = taper_ratio * self.wing_root_chord # ft
        self.mac = 2/3 * self.wing_root_chord * (1 + taper_ratio + taper_ratio**2) / (1 + taper_ratio) # ft

        # Powerplant - rubber engine sizing
        p_req = self.P_takeoff/self.num_engines
        self.engine_weight = 5.47*(p_req)**0.78 # lbs
        self.engine_len = 0.32*(p_req)**0.424 # ft
        self.engine_width = 2.7 # ft
        self.engine_height = 2.0 # ft

Note the post initialization method which computes further quantites based on the defined data. While initializing this class, the aspect ratio, wing reference area, and maximum takeoff power are **required**. Rest all parameters can be changed in class definition as desired. Below code block shows how one can instantiate the above class:

In [31]:
A = 8
S = 134 # sq ft
max_power = 595 # hp
R = 1500 # nmi

aircraft = AircraftParameters(A, S, max_power, range=R)

print(f"Aspect ratio: {aircraft.A}")
print(f"Wing span: {aircraft.b:.0f} ft")
print(f"Range: {aircraft.range} nmi")

Aspect ratio: 8
Wing span: 33 ft
Range: 1500 nmi


This concludes the parameter module, next section outlines the fuel estimation module.